<a href="https://colab.research.google.com/github/NamanChaudhary1/RAGAPP/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain huggingface_hub sentence_transformers faiss-cpu unstructured chromadb Cython tiktoken unstructured[local-inference] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26

In [17]:
import getpass
import os

if "HUGGINGFACEHUB_API_TOKEN"  in os.environ:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Provide your HUGGINGFACEHUB TOKEN")

Provide your HUGGINGFACEHUB TOKEN··········


In [7]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

pdf_folder_path = '/content/gdrive/My Drive/pdfs/'
os.listdir(pdf_folder_path)

Mounted at /content/gdrive


['traffic Guidelines.pdf']

In [9]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
loaders

In [11]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=500, chunk_overlap=0)).from_loaders(loaders)

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [18]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint
llm=HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1, max_length=512)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=index.vectorstore.as_retriever(),
                                    input_key="question")

In [20]:
chain.run('How to avoid accident?')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


" To avoid accidents while traveling by bus, follow these safety tips:\n\n1. Always check that the bus is at a complete stop before standing up to get off.\n2. Hold on to the handrails or straps when the bus is moving to keep your balance.\n3. Keep a safe distance from the bus doors when they open to avoid being hit by other passengers or vehicles.\n4. Be aware of your surroundings and the traffic conditions when getting on or off the bus.\n5. Stay seated and keep your belongings close to you to prevent theft.\n6. Avoid distracting the bus driver by talking loudly or moving around in the bus.\n7. Follow the bus company's rules and regulations for safety.\n8. In case of an emergency, know where the emergency exits are located and how to use them.\n\nBy following these safety tips, you can help ensure a safe and enjoyable bus ride."

In [31]:
import pandas as pd
def is_table_prompt(prompt):
    table_keywords = ["create table", "generate table", "table of"]
    return any(keyword in prompt.lower() for keyword in table_keywords)

def format_as_table(content):
    print("Generated content:\n", content)  # Debugging step to understand the content format
    try:
        # Assuming the content is a list of lists or similar structure
        data = [row.split() for row in content.strip().split('\n') if row]
        max_cols = max(len(row) for row in data)  # Find the maximum number of columns in the data
        if len(data) < 2:
            raise ValueError("Not enough data to form a table")

        # Ensure all rows have the same number of columns
        for i in range(len(data)):
            if len(data[i]) < max_cols:
                data[i] += [''] * (max_cols - len(data[i]))  # Fill missing columns with empty strings

        df = pd.DataFrame(data[1:], columns=data[0])  # Assuming first row is header
        return df
    except Exception as e:
        print("Error in formatting as table:", e)
        return content

def process_prompt(prompt):
    if is_table_prompt(prompt):
        table_prompt = f"Generate a table: {prompt}"
        response = chain.run({"question": table_prompt})
        return format_as_table(response)
    else:
        response = chain.run({"question": prompt})
        return response



In [26]:
prompt = "what kind of helmets should be used"
result = process_prompt(prompt)
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result)

 Full-face helmets are safer as they protect the face and temples in addition to the head. They should have a thick padding of thermocole, at least 20 mm, which must extend to the sides of the head. The shell should be rigid and have a convex shape to distribute forces from impacts and allow sliding on road surfaces. The helmet should not obstruct vision, interfere with hearing, build up dangerous temperatures inside, cause fatigue, or increase the probability of neck injuries. It is essential to wear a well-fitting helmet with a properly functioning chinstrap system.


In [32]:
prompt = "Generate a table of six-step crossing code"
result = process_prompt(prompt)
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result)

Generated content:
  I'm sorry, but the question is not clear as it is asking to generate a table of the six-step crossing code which is already provided in the text. The text itself is the table.
Error in formatting as table: Not enough data to form a table
 I'm sorry, but the question is not clear as it is asking to generate a table of the six-step crossing code which is already provided in the text. The text itself is the table.


In [33]:
prompt = "Generate any table from pdf"
result = process_prompt(prompt)
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result)

 I cannot generate a table directly from the text provided. However, if you have a PDF file with a table that you would like help extracting data from, I can guide you through the process using various tools such as Adobe Acrobat, LibreOffice Calc, or Microsoft Excel. Let me know if you have a specific PDF file and the type of data you would like to extract.


In [35]:
prompt = "Generate a title for pdf"
result = process_prompt(prompt)
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result)

 "Ensuring Pedestrian Safety: The Role of Full-Face Helmets"


In [37]:
prompt = "generate a table summarizing the key points and categories mentioned in detail"
result = process_prompt(prompt)
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result)

 | Category | Key Points |
| --- | --- |
| Children's Safety | - Children should be supervised at bus stops. <br> - Every school bus must have a teacher. <br> - Teachers should arrange for alternate buses. <br> - Regular checking of school buses. <br> - Reporting traffic rule violations. |
| Parents' Role | - Parents should observe and report violations. <br> - Participation in P.T.A. meetings. <br> - Proper care while taking children to school. <br> - Teaching children road safety. <br> - Not allowing minors to drive. <br> - Setting an example for children. |
| Drivers' Responsibility | - Obey traffic signals. <br> - Drive within speed limit. <br> - Be alert for pedestrians while turning. <br> - Avoid driving under influence or medication. |


In [38]:
prompt = "Generate a table highlighting the safety measures for two-wheeler riders, focusing on speed control and conspicuity, with explanations for each measure."
result = process_prompt(prompt)
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result)



| Safety Measures for Two-Wheeler Riders | Explanation |
| --- | --- |
| Wear Protective Gear | Wearing protective gear such as helmets, jackets, gloves, and boots can help reduce the severity of injuries in case of an accident. |
| Maintain a Safe Speed | Maintaining a safe speed is crucial for two-wheeler riders as they are more vulnerable to injuries due to the lack of protection. Riding at a speed that is within the legal limit and appropriate for the road conditions can help prevent accidents. |
| Use Headlights and Reflectors | Using headlights and reflectors can help increase the visibility of two-wheeler riders, making them more conspicuous to other road users. This can help prevent accidents caused by lack of visibility. |
| Avoid Overloading | Overloading a two-wheeler can affect its balance and handling, making it more difficult to control. This can increase the risk of accidents, especially at high speeds. |
| Keep a Safe Distance | Keeping a safe distance from other vehi

In [39]:
prompt = "Generate a table outlining the causes of pedestrian deaths in road accidents, including descriptions for each cause."
result = process_prompt(prompt)
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result)



| Causes of Pedestrian Deaths | Descriptions |
| --- | --- |
| Alcohol Impairment | Most adult pedestrian deaths occur due to this. Drivers under the influence of alcohol lose the ability to take decisive action, leading to unfortunate crashes. |
| Heavy Vehicular Traffic | In central city areas, high volume of vehicular traffic and pedestrian activity compel pedestrians to scuttle between vehicles to cross the road, putting them at high risk. |
| Absence of Adequate Space for Pedestrians | Narrow roads where vehicles and pedestrians move side by side make it difficult for pedestrians to be visible to drivers coming from behind, increasing the risk of getting hit. |
| Traffic Violations | Speeding vehicles often crush pedestrians due to the shorter time for drivers to take corrective action and the increased stopping distance, which increases both the likelihood of crashes and the severity of injuries. Red light violations by vehicle drivers can also cause pedestrian deaths. |
| Lack